# Saudi Aramco Strategy Analysis: To Invest or To Trade?

In this notebook, I will explore how a **long-term buy-and-hold strategy** compares to a **short-term Simple Moving Average (SMA) crossover strategy** for **Saudi Aramco’s stock**.

Using historical stock data from the past five years, I will:

- **Calculate the dividend-adjusted annualized return** for an investor who bought and held the stock
- **Backtest and apply walk-forward cross-validation** to determine the best short-term and long-term SMA parameters
- **Compare the two strategies** and analyze **why one outperforms the other** for this specific ticker

The goal is to help **investors and analysts** make more **informed decisions** and have **full confidence in their strategy**, regardless of the market's state.

## Importing Essential Modules & Initializing Aramco's Stock DataFrame

In [16]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import yfinance as yf

In [17]:
aramco_stock = yf.download("2222.SR", "2019-12-01", "2025-07-13", multi_level_index=False, auto_adjust=False)
aramco_stock.reset_index(inplace=True)
aramco_stock

[*********************100%***********************]  1 of 1 completed


,Date,Adj Close,Close,High,Low,Open,Volume
0,2019-12-11,24.860811,29.090908,29.090908,29.090908,29.090908,38289394
1,2019-12-12,25.990849,30.413223,31.983471,29.752066,31.983471,505692621
2,2019-12-15,26.414612,30.909090,30.991735,30.413223,30.661158,98349281
3,2019-12-16,26.838373,31.404959,31.487602,30.991735,30.991735,105973907
4,2019-12-17,26.661808,31.198347,31.528925,30.991735,31.446280,142672245
...,...,...,...,...,...,...,...
1386,2025-07-06,24.870001,24.870001,24.889999,24.549999,24.780001,9678871
1387,2025-07-07,25.080000,25.080000,25.139999,24.780001,24.870001,11038097
1388,2025-07-08,25.100000,25.100000,25.120001,24.980000,25.059999,11042534
1389,2025-07-09,24.879999,24.879999,25.120001,24.820000,25.080000,7350422


## Buy-And-Hold Performance

**Firstly**, let's calculate the **average annualized return** for an investor who **bought Saudi Aramco stock five years ago and held it without making any trades**. For simplicity, let's assume the **dividend yield remained constant at its 5.39% average** and calculate the returns over the period from **July 1, 2020 to July 1, 2025**. Let's also assume that the **yields from capital gains and dividends are compounded annually**.

In [26]:
STARTING_CASH = 100
DIVIDEND_MULTIPLIER = 1.0539
START_DATE = "2020-07-01"
END_DATE = "2025-07-01"

# calculatte annualized growth with dividends
shares_owned = STARTING_CASH / aramco_stock["Open"][aramco_stock["Date"] == START_DATE].iloc[0]
final_shares_price = shares_owned * aramco_stock["Open"][aramco_stock["Date"] == END_DATE].iloc[0]
compound_annual_growth_multiplier = (final_shares_price / STARTING_CASH) ** (1/5)
compound_annual_growth_multiplier_with_dividends = compound_annual_growth_multiplier * DIVIDEND_MULTIPLIER

print(f"On {START_DATE}, {STARTING_CASH} SAR buys {shares_owned:.2f} shares of Saudi Aramco stock.")
print(f"On {END_DATE}, these shares are worth {final_shares_price:.2f} SAR, showing a compound annual growth rate of {(compound_annual_growth_multiplier - 1) * 100:.2f}%.")
print(f"With dividends reinvested at a yield of {(DIVIDEND_MULTIPLIER - 1) * 100:.2f}%, the finalized annual return is {(compound_annual_growth_multiplier_with_dividends - 1) * 100:.2f}%.")

On 2020-07-01, 100 SAR buys 3.73 shares of Saudi Aramco stock.
On 2025-07-01, these shares are worth 90.68 SAR, showing a compound annual growth rate of -1.94%.
With dividends reinvested at a yield of 5.39%, the finalized annual return is 3.35%.


As can be observed, the **annualized return rate** for someone who **bought and held without trading** is **3.35%**, a **moderate return** for a passive strategy. Let's now dig deeper into the **SMA crossover strategy** to see if this return can be **outperformed by a more active trading approach**.

## Simple Moving Average (SMA) Crossover Performance

The way this strategy works is by calculating a **short-term** and **long-term SMA** of the stock price each day based on the **opening price**. A **buy position** is taken when the short-term SMA crosses **above** the long-term SMA, and a **sell position** is taken when it crosses **below**. For simplicity, let's assume **no brokerage fees** or **bid/ask spread**, and the strategy will always **buy or sell using 100% of the available capital**. Let's also assume **fractional shares are allowed**.

Before we **optimize the returns** of this strategy by tuning the **short-term** and **long-term SMA parameters** using **grid search** and **walk-forward cross-validation**, let's first analyze a few statistics to ensure my results **make numerical sense**.

Firstly, let's calculate the **expected value of the annual yield** if an investor were to select the **short-term** and **long-term SMA parameters at random** from a **uniform distribution of possible combinations**. We will also **determine the best parameters** for the short-term and long-term SMA durations.

In [130]:
short_term_values = range(10, 51, 5)
long_term_values = range(60, 141, 10)

annual_yields = []
maximum_yield = -float("inf")
maximum_params = (None, None)

for short_term_param in short_term_values:
    for long_term_param in long_term_values:
        current_cash = STARTING_CASH
        current_shares = 0

        # initial dataframe adjusted for the start and end dates for easier indexing
        aramco_stock_date_adjusted = aramco_stock[
            (aramco_stock["Date"] >= START_DATE) & (aramco_stock["Date"] <= END_DATE)
        ]

        # short-term and long-term series of the simple moving averages
        short_term_sma = aramco_stock["Open"].rolling(short_term_param).mean()[
            (aramco_stock["Date"] >= START_DATE) & (aramco_stock["Date"] <= END_DATE)
        ]
        long_term_sma = aramco_stock["Open"].rolling(long_term_param).mean()[
            (aramco_stock["Date"] >= START_DATE) & (aramco_stock["Date"] <= END_DATE)
        ]

        # determine if the initial position should be buy or sell
        if short_term_sma.iloc[0] > long_term_sma.iloc[0]:
            current_shares = current_cash / aramco_stock_date_adjusted["Open"].iloc[0]
            current_cash = 0

        # check for a crossover and buy or sell accordingly
        for i in range(1, len(short_term_sma)):
            if short_term_sma.iloc[i] > long_term_sma.iloc[i] and short_term_sma.iloc[i - 1] < long_term_sma.iloc[i - 1]:
                current_shares = current_cash / aramco_stock_date_adjusted["Open"].iloc[i]
                current_cash = 0
            elif short_term_sma.iloc[i] < long_term_sma.iloc[i] and short_term_sma.iloc[i - 1] > long_term_sma.iloc[i - 1]:
                current_cash = current_shares * aramco_stock_date_adjusted["Open"].iloc[i]
                current_shares = 0

        final_yield = max(current_cash, current_shares * aramco_stock_date_adjusted["Open"].iloc[-1]) / 100
        
        if final_yield > maximum_yield:
            maximum_yield = final_yield
            maximum_params = (short_term_param, long_term_param)
        
        annual_yields.append(final_yield)

cumulative_yield = np.mean(annual_yields)
annualized_yield = cumulative_yield ** (1/5)

print(f"The expected yield for an investor following an SMA crossover strategy with parameters picked at random from " +
      f"{START_DATE} to {END_DATE} is {(cumulative_yield - 1) * 100:.2f}%, or an average annual yield of {(annualized_yield - 1) * 100:.2f}%.")

print(f"The greatest yield occured with a parameter pair of {maximum_params[0]} days for the short-term SMA and {maximum_params[1]} days for the long-term SMA.")
print(f"At those parameters, the cumulative yield is {(maximum_yield - 1) * 100:.2f}% and the annualized yield is {(maximum_yield ** (1/5) - 1) * 100:.2f}%.")

The expected yield for an investor following an SMA crossover strategy with parameters picked at random from 2020-07-01 to 2025-07-01 is 13.14%, or an average annual yield of 2.50%.
The greatest yield occured with a parameter pair of 30 days for the short-term SMA and 80 days for the long-term SMA.
At those parameters, the cumulative yield is 36.56% and the annualized yield is 6.43%.
